In [1]:
import numpy as np
import torch
import random

In [10]:
target = torch.tensor([[0., 0., 1., 1., 1.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 1.]])
print(target.shape)
target

torch.Size([5, 5])


tensor([[0., 0., 1., 1., 1.],
        [0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 1.],
        [1., 1., 0., 0., 1.]])

In [12]:
np.nonzero(target.numpy())

(array([0, 0, 0, 1, 1, 2, 3, 3, 4, 4, 4]),
 array([2, 3, 4, 3, 4, 4, 0, 4, 0, 1, 4]))

In [13]:
target = torch.zeros((5,5))
print(target.shape)
target

torch.Size([5, 5])


tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [14]:
target = torch.ones((5,5))
print(target.shape)
target

torch.Size([5, 5])


tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [86]:
# target = np.zeros((5,5))
# target[0][0]=1
# print(target.shape)
# target

In [195]:
n=2
target = torch.tensor(target)
triplet_indices = get_triplet_indices(target, n)
triplet_indices

/var/folders/5d/td2lpmcj3vq5wsprrdr42r9w0000gn/T/ipykernel_43943/516819316.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target)


(array([[4, 4],
        [3, 0]]),
 array([[3, 3],
        [2, 1]]),
 array([[1, 4],
        [0, 3]]))

In [235]:
output = torch.rand((2,5,5))
output

tensor([[[0.6375, 0.5182, 0.4396, 0.0555, 0.0057],
         [0.5666, 0.8391, 0.4668, 0.0356, 0.8924],
         [0.3725, 0.6685, 0.8238, 0.8782, 0.9070],
         [0.2593, 0.5689, 0.4267, 0.0236, 0.8909],
         [0.9445, 0.1930, 0.5347, 0.1236, 0.3778]],

        [[0.4523, 0.5304, 0.7748, 0.9978, 0.2531],
         [0.9391, 0.7552, 0.1014, 0.0545, 0.8908],
         [0.0469, 0.3538, 0.1531, 0.5613, 0.4489],
         [0.0234, 0.8388, 0.2468, 0.6098, 0.6700],
         [0.7683, 0.1252, 0.7088, 0.4645, 0.6962]]])

In [252]:
def get_output_triplets(output, triplet_indices):
#     print(output.shape)
    
    # unpack
    anchors, negatives, positives = triplet_indices
    
    output_anchors = []
    output_negatives = []
    output_positives = []
    
    for j,k in anchors:
        output_anchor = output[:,j,k]
        output_anchors.append(output_anchor)
        
    for j,k in negatives:
        output_negative = output[:,j,k]
        output_negatives.append(output_negative)
        
    for j,k in positives:
        output_positive = output[:,j,k]
        output_positives.append(output_positive)

    # convert lists to tensors
    tensor_anchors = torch.stack(output_anchors).T
    tensor_positives = torch.stack(output_negatives).T
    tensor_negatives = torch.stack(output_positives).T

    return tensor_anchors, tensor_positives, tensor_negatives

In [253]:
tensor_triplets = get_output_triplets(output, triplet_indices)
tensor_triplets

(tensor([[0.3778, 0.2593],
         [0.6962, 0.0234]]),
 tensor([[0.0236, 0.6685],
         [0.6098, 0.3538]]),
 tensor([[0.8924, 0.0555],
         [0.8908, 0.9978]]))

In [329]:
def triplet_loss(tensor_triplets, margin=0.2):
    
    # unpack
    tensor_anchors,tensor_negatives, tensor_positives = tensor_triplets
    print(f'tensor_anchors:\n{tensor_anchors}\n')
    print(f'tensor_negatives:\n{tensor_negatives}\n')
    print(f'tensor_positives:\n{tensor_positives}\n')
    
    diff_anchor_positives = tensor_anchors - tensor_positives
    norm_anchor_positives = torch.linalg.norm(diff_anchor_positives)
    print(f'norm_anchor_positives:{norm_anchor_positives}')
    
    diff_anchor_negatives = tensor_anchors - tensor_negatives
    norm_anchor_negatives = torch.linalg.norm(diff_anchor_negatives)
    print(f'norm_anchor_negatives:{norm_anchor_negatives}')
    
    option1 = norm_anchor_positives - norm_anchor_negatives + margin
    option2 = 0.0
    
    loss = max(option1, option2)
    return float(loss)

In [347]:
loss = triplet_loss(tensor_triplets)
print(f'loss:{loss}')

tensor_anchors:
tensor([[0.3778, 0.2593],
        [0.6962, 0.0234]])

tensor_negatives:
tensor([[0.0236, 0.6685],
        [0.6098, 0.3538]])

tensor_positives:
tensor([[0.8924, 0.0555],
        [0.8908, 0.9978]])

norm_anchor_positives:1.137434959411621
norm_anchor_negatives:0.6399896740913391
loss:0.697445273399353


In [345]:
# def triplet_loss2(tensor_triplets, margin=0.2):
 
#     # unpack
#     tensor_anchors,tensor_negatives, tensor_positives = tensor_triplets
    
#     diff_anchor_positives = torch.cdist(tensor_anchors,tensor_positives,p=2.0)
#     print(f'diff_anchor_positives:\n{diff_anchor_positives}\n')
    
#     diff_anchor_negatives = torch.cdist(tensor_anchors,tensor_negatives,p=2.0)
#     print(f'diff_anchor_negatives:\n{diff_anchor_negatives}\n')
    
#     option1 = diff_anchor_positives - diff_anchor_negatives + margin
#     print(f'option1:\n{option1}\n')
    
#     option2 = 0.0
    
#     loss = max(option1, option2)
#     return float(loss)

In [348]:
# loss_alt = triplet_loss2(tensor_triplets)
# print(f'loss_alt:{loss_alt}')

In [341]:
targets = torch.stack([target,target])
print(targets.shape)
targets

torch.Size([2, 5, 5])


tensor([[[0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [1., 1., 0., 0., 1.]],

        [[0., 0., 1., 1., 1.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 0., 1.],
         [1., 0., 0., 0., 1.],
         [1., 1., 0., 0., 1.]]])

In [342]:
def loss(output, targets, n=1, margin=0.5):
    batch_loss = 0.0
    
    # for each output in the mini-batch, calculate triplet_loss
    for i in range(len(output)):
        target_i = targets[i]
        output_i = output[i]
#         print(output_i.shape)
        if len(output_i.shape) < 3:
            # output tensor must have at least 3 dimensions for function to work
            output_i = torch.unsqueeze(output_i, 0)
#         print(output_i.shape)
        
        
        
        # pixel indices
        triplet_indices_i = get_triplet_indices(target_i, n)
        print(triplet_indices_i)
        
        # output pixels
        tensor_triplets_i = get_output_triplets(output_i, triplet_indices_i)
#         print(tensor_triplets_i)
        
        # calculate loss
        loss_i = triplet_loss(tensor_triplets_i, margin)
        batch_loss += loss_i
        
    return batch_loss

In [302]:
loss(output, targets)

0.7193681001663208